In [1]:
import requests
import sqlite3
import time

In [2]:
base_url = "https://api.jikan.moe/v4/anime"

In [2]:
conn = sqlite3.connect("anime_data.db")
cursor = conn.cursor()

# Crea una tabla para almacenar los datos si no existe
cursor.execute("""
    CREATE TABLE IF NOT EXISTS animes (
        id INTEGER PRIMARY KEY,
        title TEXT,
        url TEXT
    )
""")
conn.commit()


In [4]:
data = [
    {
        "id": 1,
        "title": "Naruto",
        "url": "https://api.jikan.moe/v4/anime/1"
    }
]
conn = sqlite3.connect("anime_data.db")
cursor = conn.cursor()
#insert on conlict update 
cursor.executemany("""
    INSERT INTO animes (id, title, url) VALUES (:id, :title, :url)
    ON CONFLICT(id) DO UPDATE SET title=:title, url=:url
""", data)
conn.commit()

In [12]:
requests_per_minute = 60
requests_per_second = 3

In [13]:
def fetch_and_store_data(page):
    url = f"{base_url}?page={page}"
    response = requests.get(url)
    data = response.json()
    animes = []
    for item in data["data"]:
        obj= {
            "id": item["mal_id"],
            "title": item["title"],
            "url": item["url"]
        }
        anime_id = item["mal_id"]
        title = item["title"]
        url = item["url"]
        animes.append(obj)

        # # Verifica si el anime ya existe en la base de datos
        cursor.execute("SELECT id FROM animes WHERE id = ?", (anime_id,))
        existing_row = cursor.fetchone()

        if not existing_row:
            # Inserta el anime en la base de datos si no existe
            cursor.execute("INSERT INTO animes (id, title,url) VALUES (?, ?, ?)", (anime_id, title,url))
            conn.commit()

    return animes


In [ ]:
total_pages = 25481 // 25  # Calcula el número total de páginas
for page in range(1, total_pages + 1):
    data = fetch_and_store_data(page)
    
    # Calcula el tiempo de espera entre las solicitudes para respetar el límite por segundo
    if page % requests_per_second == 0:
        time.sleep(1)

    # Calcula el tiempo de espera para respetar el límite por minuto
    if page % (requests_per_minute // requests_per_second) == 0:
        time.sleep(60)

# Cierra la conexión a la base de datos
conn.close()